# Discovery - shallow dive

This notebook examines the relationships between the different tables used by the Reporting metadata, two at a time.

We start with collections that allow us to explore the cardinality of references.



## This shows the main inter-table links
These were determined by inspecting the schema Primary Keys and Foreign keys or implied keys.

![Overview](../.media/Overview02.svg)

The detailed view, including all column types were also dumped from the Information_Schema and although not directly used here, links are included for reference:

(links to [Gropu A](../.media/GroupA.pdf) and [Group B](../.media/GroupB.pdf) for detailed reference)


Examining the overview gives us a list of tables that reference one another:

| a | b | c |d |e |f | g |
|-- | -- | -- | -- | -- | -- | --
|    |ReportFileLog  13->         |     ReportLogs | | | |
| |ReportActivityLog 12->         |     ReportLogs | 
|                  |            |     ReportLogs | 14->                  ReportTransmissionLogs
|ReportSchedule 9-> |Reports <-8  |     ReportLogs|
|                 |Reports 10->  |     ReportTransmitterConfigLinks  | 19-> ReportTransmitterConfigs|
|                 |Reports 15->  |     ReportMethods | |||
|                 |            |     ReportInstances  18->          |    ReportTimeRanges
|                 |Reports 11->  |     ReportInstances  7->          |    ReportDefinitions 20->        |  ReportTypes 1->         ReportCategory
|                 |            |ReportDefinitionCustomAttributes  | 21-> ReportDefinitions           | |
|                 |            |     ReportDefinitionColumns  6->  |    ReportDefinitions           | |
|                 |            |     ReportDefinitionColumns  0->  |    ReportAggregateTypes        ||
|                 |            |     ReportDefinitionColumns  2->  |    ReportColumns 3->            |   ReportDataTypes    | 
|                 |            |                                  |    ReportDataTypeOperators 4->  |   ReportDataTypes|
|                 |            |                                  |    ReportDataTypeOperators 16->  |   ReportOperatorTypes|
|                 |            |     ReportDefinitionColumns  5->  |    ReportDataTypeOverride      |                        |               
|                 |            |     ReportDefinitionColumns  -17>  |    ReportSearchableTermTypeId  ||

## Navigability

Based on the above, we can plat the navigable paths (arrows) and establish the hierarchy

<pre>
                                               /-(05)->R.DataTypes--(04)->R.DataTypeOperators--(16)->R.OperatorTypes
                                              /  /-(02)->R.Columns
                                             /  /  /-(00)->R.AggregateTypes
                         /<-(06)--R.DefinitionColumns--(17)->R.SearchableTermTypes               
                        /  /<-(21)--R.DefinitionCustomAttributes     
        /--(07)->R.Definitions--(20)->R.Types--(01)->R.Categories
R.Instances--(18)->R.TimeRanges
        \<-(11)-Reports<-(10)--R.TransmitterConfigLinks--(19)->R.TransmitterConfigs
              \  \  \<-(09)--R.Schedule
               \  \--(15)-> R.Methods    
                \<-(08)--R.Logs <-(12)-R.ActivityLog                 
                          \  \<-(13)-R.FileLog
                           \<-(14)-R.TransmissionLogs
</pre>   

Clearly the upper half contains the **structural** information and the lower half the **run-time** data.

## Static and Live sequences

These hierarchies can be grouped for processing 

> 18+11 < (10 < 19)+09+15+(8 < 12+13+14)
> 
> 07 < (20 < 01)+21+(06 <17+00+02+(05 < 04 < 16))

or to put it another way,

> =18/10/19
> +11+09
>    +15
>    +08/12
>       +13
>       +14
> 
> =07/20/01
>    +21
>    +06/17
>       +08
>       +02
>       +05/04/16


# The Raw library contains some code to allow us to reflect these relationships.

We first load the libraries. If displaying results with Razor, we will need to restart to overcome the dictionary limitations, but when we have meaningful output we can instead write the output to html files which can then be launched using #!pwsh.

### Set up the library code using Nuget

> Set up the local nuget path and any needed packages here.

In [1]:
// load packages - your local nuget folder may vary
#i "file:///n:/nix-nuget/"
#r "nuget: nix-library-raw"  
#r "nuget: Microsoft.Data.SqlClient, 5.1.5"
// you may want the razor interactive support too!
#r "nuget: RazorInteractive"

Restore sources file:///n:/nix-nuget/ Installed Packages Microsoft.Data.SqlClient, 5.1.5 nix-library-raw, 1.0.18 RazorInteractive, 1.2.0

Loading extensions from `C:\Users\NickMajor\.nuget\packages\razorinteractive\1.2.0\lib\net8.0\RazorInteractive.dll`

Renders the code block as Razor markup in dotnet-interactive notebooks. 
 This extension adds a new kernel that can render Razor markdown. 
 All C# and F# variables are available in the @Model property.

### Json Sql Client

> This part of the library allows us to generate and/or execute Sql queries and return json with divergent (1:many) relationships. 
> 
> We typically only want a sample of a few records, so the `top` syntax is included in the generated queries.
> 
> First set up the JsonSqlClient

In [2]:
// some housekeeping needed to set up the library
using Nix.Library.Raw;
string connectionString  = System.Environment.GetEnvironmentVariable("NIX_DB_DG");
JsonSqlClient sql = new JsonSqlClient(connectionString);


### Sql generation

> Every link between two tables is fully defined by knowing the fully qualified names of the linked columns.
> 
> When linking these in a single query, each row of the outer (primary key) table will contain 0 or more instances of the inner (foreign key) table.
> 
> The `GenerateNestedQuery` method requires the fully qualified names of the two tables (schema.table.column) from which it can generate query which returns all columns with `for json path` nesting to accommodate the divergent cardinality.
>
> The column names are determined through reflection, using the `Information_schema`.

In [3]:
// To feed the generator, we will use an array of string[]s
var data = new string[][] {
new string[] {"rpt.ReportAggregateTypes.ReportAggregateTypeId", "rpt.ReportDefinitionColumns.ReportAggregateTypeId" }, //0
new string[] {"rpt.ReportCategories.ReportCategoryId", "rpt.ReportTypes.ReportCategoryId" }, //1
new string[] {"rpt.ReportColumns.ReportColumnId", "rpt.ReportDefinitionColumns.ReportColumnId" }, //2
new string[] {"rpt.ReportDataTypes.ReportDataTypeId", "rpt.ReportColumns.ReportDataTypeId" }, //3
new string[] {"rpt.ReportDataTypes.ReportDataTypeId", "rpt.ReportDataTypeOperators.ReportDataTypeId" }, //4
new string[] {"rpt.ReportDataTypes.ReportDataTypeId", "rpt.ReportDefinitionColumns.ReportDataTypeIdOverride" }, //5
new string[] {"rpt.ReportDefinitions.ReportDefinitionId", "rpt.ReportDefinitionColumns.ReportDefinitionId" }, //6
new string[] {"rpt.ReportDefinitions.ReportDefinitionId", "rpt.ReportInstances.ReportDefinitionId" }, //7
new string[] {"rpt.Reports.ReportId", "rpt.ReportLogs.ReportId" }, //8
new string[] {"rpt.Reports.ReportId", "rpt.ReportSchedule.ReportId"  }, //9
new string[] {"rpt.Reports.ReportId", "rpt.ReportTransmitterConfigLinks.ReportId" }, //10
new string[] {"rpt.ReportInstances.ReportInstanceId", "rpt.Reports.ReportInstanceId" }, //11
new string[] {"rpt.ReportLogs.ReportLogId", "rpt.ReportActivityLog.ReportLogId" }, //12
new string[] {"rpt.ReportLogs.ReportLogId", "rpt.ReportFileLog.ReportLogId" }, //13
new string[] {"rpt.ReportLogs.ReportLogId", "rpt.ReportTransmissionLogs.ReportLogId" }, //14
new string[] {"rpt.ReportMethods.Id", "rpt.Reports.ReportMethodId" }, //15
new string[] {"rpt.ReportOperatorTypes.ReportoperatorTypeId", "rpt.ReportDataTypeOperators.ReportOperatorTypeId" }, //16
new string[] {"rpt.ReportSearchableTermTypes.ReportSearchableTermTypeId", "rpt.ReportDefinitionColumns.ReportSearchableTermTypeId" }, //17
new string[] {"rpt.ReportTimeRanges.ReportTimeRangeId", "rpt.ReportInstances.TimePeriod" }, //18
new string[] {"rpt.ReportTransmitterConfigs.ReportTransmitterConfigId", "rpt.ReportTransmitterConfigLinks.ReportTransmitterConfigId"}, //19
new string[] {"rpt.ReportTypes.ReportTypeId", "rpt.ReportDefinitions.ReportTypeId" }, //20
new string[] {"rpt.ReportDefinitions.ReportDefinitionId", "rpt.ReportDefinitionCustomAttributes.ReportDefinitionId"} //21
};

var queries = new List<string>();

foreach(var line in data)
{
    var s = sql.GenerateNestedQuery(line[0], line [1]);
    queries.Add(s);
}
// remedial:

$"{queries.Count()} queries were generated. First example:\n{queries[0]}".Display();

22 queries were generated. First example:
 select top (10) [ReportAggregateTypeId],[ConditionName],[Operator],[CreatedDate],[ModifiedDate], (select [ReportDefinitionColumnId],[ReportDefinitionId],[ReportColumnId],[Order],[IsDefault],[IsFilterable],[CreatedDate],[ModifiedDate],[ReportAggregateTypeId],[IsForDateRange],[IsRequired],[LocalizationKeyOverride],[IsVisible],[ReportSearchableTermTypeId],[ReportDataTypeIdOverride],[IsForSegmentJoin] from rpt.ReportDefinitionColumns where rpt.ReportAggregateTypes.ReportAggregateTypeId=rpt.ReportDefinitionColumns.ReportAggregateTypeId for json path, root('ReportAggregateTypes'), INCLUDE_NULL_VALUES) as ReportDefinitionColumns from rpt.ReportAggregateTypes order by rpt.ReportAggregateTypes.ReportAggregateTypeId desc  for json path, INCLUDE_NULL_VALUES

In [ ]:
/*
 // to examine the sql
int ix = 0;
foreach (var q in queries)
($"{ix++}: {q}").Display();
*/

### Json Pages

> Running the sql queries using the `JsonSingleUsingQuery` method returns a json object representing the data.
> 
> Other functions are available to 
> 
> - return multiple result sets, 
> - query using stored procedures,
> - return data as strings not json.
> 
> These provide flexibility, depending on the data needed. 
>
> For our purposes we only need a single data set per query, and need json to feed to the next step.

In [4]:
var jsonPages = new List<string>();

foreach (var query in queries)
{
    //Console.Write($" {index++}");
    string j = sql.JsonSingleUsingQuery(query, false);
    jsonPages.Add(j);
}
$"{jsonPages.Count()} json pages were generated. First example:\n{jsonPages[0]}".Display();


22 json pages were generated. First example:
[{"ReportAggregateTypeId":5,"ConditionName":"Sum","Operator":"SUM('%')","CreatedDate":"2021-07-16T22:34:53.2027056","ModifiedDate":null,"ReportDefinitionColumns":{"ReportAggregateTypes":[{"ReportDefinitionColumnId":8,"ReportDefinitionId":1,"ReportColumnId":8,"Order":8,"IsDefault":true,"IsFilterable":false,"CreatedDate":"2021-07-16T22:34:53.2027056","ModifiedDate":null,"ReportAggregateTypeId":5,"IsForDateRange":false,"IsRequired":false,"LocalizationKeyOverride":null,"IsVisible":true,"ReportSearchableTermTypeId":null,"ReportDataTypeIdOverride":null,"IsForSegmentJoin":false},{"ReportDefinitionColumnId":21,"ReportDefinitionId":3,"ReportColumnId":21,"Order":11,"IsDefault":true,"IsFilterable":false,"CreatedDate":"2021-07-16T22:34:53.2027056","ModifiedDate":null,"ReportAggregateTypeId":5,"IsForDateRange":false,"IsRequired":false,"LocalizationKeyOverride":"AssignmentsCompleted","IsVisible":true,"ReportSearchableTermTypeId":null,"ReportDataTypeIdOver

### Saving

> To preserve the data, useful for snapshots and offline working, the `Nest` class that provides the ability to handle nested objects includes static methods to save and retrieve files.
> 
> Typically we also save the sql files so that we can edit, adding filters or commenting out fields as needed.

In [5]:
int ix = 0;
foreach (var q in jsonPages)
{
    Nest.ToFile(q, $"../.data/json{ix++:00}.json");
}

### Rendering

> The choice of json as an output and storage format makes it simple to use html / css able to display the data in a variety of forms.
>
> The Nest class contains some rendering code which takes the 

In [6]:
int index = 0;
var output = new List<string>();
foreach (var jsonPage in jsonPages)
{
    var title = $"{data[index][0]} -> {data[index][1]}";
    var path = $"../.Html/page{index++:00}.html";
    Nest.ToFile(Nest.RenderPageStepping(jsonPage, " ", 10, title), path);
    output.Add(path); 
}

### Show the data

> This example used the `RenderPageStepping`  method to produce stand-alone html with embedded css and javascript.
> The display allows you to step through multiple values resulting from the nesting.

In [7]:
// Launch all in the browser
using System.Diagnostics;
foreach (string s in output)
{   
    Nest.RunFile(s); 
}